In [1]:
import numpy as np
from qutip import Bloch
import pickle
import os
import shutil
import imageio.v2 as imageio
from IPython.display import Image, display
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [2]:

sz = np.array(([[1,0], [0,-1]]), dtype=complex); sx = np.array(([[0,1],[1,0]]), dtype=complex); sy = np.array(([[0,-1j],[1j,0]]), dtype=complex) 


In [4]:
# -----------------------
# Directory and file name
# -----------------------
input_path = "../Results/Data/"
input_file = os.path.join(input_path, "results_General_prova_diff.pkl") # change for different plot

output_path = "../Results/Bloch_Sphere"
gif_path = os.path.join(output_path, 'General_prova_prob.gif')   # change : name gif
os.makedirs(output_path, exist_ok=True)

# --------------------
# Temporary Directory
# --------------------
temp_dir = os.path.join(output_path, 'temp_bloch_frames')

if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir)
    

In [ ]:
# ------------
# Load Results
# ------------
with open(input_file, 'rb') as f:
    results = pickle.load(f)

# Extract and sample data (every 10 steps)
sample_step = 10 

v_x = results[0.01][1000]['trajectory_wf']['bloch']['samples_x'][::sample_step, 0]
v_y = results[0.01][1000]['trajectory_wf']['bloch']['samples_y'][::sample_step, 0]
v_z = results[0.01][1000]['trajectory_wf']['bloch']['samples_z'][::sample_step, 0]
avg_x = results[0.01][1000]['trajectory_wf']['bloch']['avg_x'][::sample_step]
avg_y = results[0.01][1000]['trajectory_wf']['bloch']['avg_y'][::sample_step]
avg_z = results[0.01][1000]['trajectory_wf']['bloch']['avg_z'][::sample_step]


In [ ]:
# ================
# Frame generation
# ================

n_steps = len(v_x) # number of time steps

sphere = Bloch()
sphere.view = [-60, 30]

for i in range(n_steps):
    sphere.clear()
    
    # Add points with fading effect
    for j in range(i + 1):    # j=0 fisrts point, till last point in time i+1, with fading exponentialy increasing
        sphere.add_points([v_x[j], v_y[j], v_z[j]], alpha = np.exp((j - i) / 15))

    # Add average vector
    sphere.add_vectors([avg_x[i], avg_y[i], avg_z[i]])
            
    # Configuration
    sphere.point_color = 'b'
    sphere.point_marker = 'o'
    sphere.point_size = [25]

    # Build up
    sphere.make_sphere()
    
    # Saving
    frame_name = f'bloch_{str(i).zfill(len(str(n_steps)))}.png'
    frame_path = os.path.join(temp_dir, frame_name)
    sphere.fig.savefig(frame_path, dpi=80, bbox_inches='tight')


In [ ]:
# ==========
# Create GIF 
# ==========

filenames = sorted([os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith('.png')])

with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:   # duration of gif steps
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Clean up temporary files
shutil.rmtree(temp_dir)

In [6]:
# =============
# Visualization
# =============
display(Image(filename=gif_path))